In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content'

In [ ]:
!chmod 600 /content/kaggle.json

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle

In [ ]:
!kaggle competitions download -c plant-seedlings-classification

In [ ]:
!unzip \*.zip && rm *.zip

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
from PIL import Image
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [ ]:
test_dir="/content/test"
train_dir="/content/train"

In [ ]:
train_dir_Black_grass = train_dir + '/Black-grass'
train_dir_Charlock = train_dir + '/Charlock'
train_dir_Cleavers = train_dir + '/Cleavers'
train_dir_Common_Chikweed = train_dir + '/Common Chikweed'
train_dir_Common_wheat = train_dir + '/Common Wheat'
train_dir_Fat_hen = train_dir + '/Fat Hen'
train_dir_Loose = train_dir + '/Loose Silky-bent'
train_dir_Maize = train_dir + '/Maize'
train_dir_Scentless = train_dir + '/Scentless Mayweed'
train_dir_Shepherds = train_dir + '/Shepherds Purse'
train_dir_Small = train_dir + '/Small-flowered Cranesbill'
train_dir_Sugar = train_dir + '/Sugar beet'


In [ ]:
print('number of training images classes - ',len(os.listdir(train_dir)))
print('number of testing images - ',len(os.listdir(test_dir)))

In [ ]:
data_generator = ImageDataGenerator(rotation_range=45,zoom_range=0.2,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15,horizontal_flip=True,vertical_flip=True,fill_mode="nearest")

In [ ]:
batch_size = 32
training_data = data_generator.flow_from_directory(directory = train_dir,
                                                   target_size = (64, 64),
                                                   batch_size = batch_size,
                                                   class_mode = 'categorical')

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation , BatchNormalization

In [ ]:
training_data.image_shape

In [ ]:
len(set(training_data.classes))

In [ ]:
model = Sequential()
model.add(Dense(64,activation='relu',input_shape = training_data.image_shape))
model.add(BatchNormalization())

model.add(Conv2D(64, kernel_size = (4,4),activation='relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(32, kernel_size = (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(rate = 0.25))

model.add(Conv2D(32, kernel_size = (2, 2), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(rate = 0.25))

model.add(Flatten())
model.add(Dense(units = 32, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(rate = 0.1))

model.add(Dense(units = len(set(training_data.classes)), activation = 'softmax'))

In [ ]:
from keras.optimizers import Adam
opt = Adam(learning_rate=0.001,beta_1=0.9,beta_2=0.999,epsilon=1e-07,amsgrad=False)
model.compile(optimizer = opt, loss = 'CategoricalCrossentropy', metrics = ['accuracy'])

In [ ]:
model.summary()

In [ ]:
fitted_model = model.fit(training_data,
                        epochs = 25)

In [ ]:
plt.plot(fitted_model.history['loss'], color='b')
plt.plot(fitted_model.history['accuracy'], color='r')
plt.show()